In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal

In [2]:
data = np.loadtxt('./winery-multivariate/wine.data.txt', delimiter=',')
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']

np.random.seed(0)
perm = np.random.permutation(178)
train_size = 130
feature_size = 14
trainx = data[perm[0:train_size],1:feature_size]
trainy = data[perm[0:train_size],0]
testx = data[perm[train_size:], 1:feature_size]
testy = data[perm[train_size:],0]

In [3]:
print(trainx.shape, trainy.shape, testx.shape, testy.shape)

(130, 13) (130,) (48, 13) (48,)


In [4]:
def fit_generative_model(x, y):
    num_class = 3
    k = num_class
    d = (x.shape)[1]
    mu = np.zeros((k+1, d))
    sigma = np.zeros((k+1, d, d))
    pi = np.zeros(k+1)
    for label in range(1, k+1):
        indices = (y==label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [5]:
mu, sigma, pi = fit_generative_model(trainx, trainy)

In [16]:
print(mu.shape, sigma.shape, pi.shape)
print(mu[1:4,:])
sub_feature = [2,4,6]
sub_mu = mu[1:4, sub_feature]
sub_sigma = sigma[1:4, sub_feature, sub_feature]
print('mu subset = \n', sub_mu)
print('cov subset = \n', sub_sigma)

(4, 13) (4, 13, 13) (4,)
[[1.37853488e+01 2.02232558e+00 2.42790698e+00 1.68813953e+01
  1.05837209e+02 2.85162791e+00 2.99627907e+00 2.89069767e-01
  1.93069767e+00 5.63023256e+00 1.06232558e+00 3.16674419e+00
  1.14190698e+03]
 [1.23109259e+01 1.91925926e+00 2.22703704e+00 1.96759259e+01
  9.58333333e+01 2.30129630e+00 2.10907407e+00 3.47407407e-01
  1.65722222e+00 3.18370370e+00 1.04129630e+00 2.76666667e+00
  5.25981481e+02]
 [1.31596970e+01 3.37727273e+00 2.40090909e+00 2.08939394e+01
  9.90303030e+01 1.67606061e+00 7.57272727e-01 4.59696970e-01
  1.20848485e+00 7.38242421e+00 6.86666667e-01 1.67484848e+00
  6.35909091e+02]]
mu subset = 
 [[  2.42790698 105.8372093    2.99627907]
 [  2.22703704  95.83333333   2.10907407]
 [  2.40090909  99.03030303   0.75727273]]
cov subset = 
 [[3.67746890e-02 1.18461871e+02 1.52409410e-01]
 [1.12302332e-01 3.28509259e+02 5.68697291e-01]
 [2.67597796e-02 1.25423324e+02 7.37531680e-02]]


In [21]:
def test_model(mu, sigma, pi, features, tx, ty):
    num_class = 3
    sub_mu = mu[1:num_class+1, features]
    sub_sigma = sigma[1:num_class+1, features, features]
    for label in range(1, num_class+1):
        p1 = ((tx - sub_mu).T * (np.linalg.inv(sub_sigma)) * (tx - sub_mu)) * (-0.5)